# Sistemas inteligentes. Aplicaciones

## Práctica 1. Análisis de sentimientos

En esta práctica vas a implementar dos soluciones a un problema de análisis de sentimientos. El problema concreto se centra en identificar si un tweet tiene un sentimiento positivo o negativo.

El conjunto de tweets con el que vas a trabajar está disponible dentro de la librería nltk (Natural Language Toolkit). Esta librería es una de las más utilizadas para el procesamiento del lenguaje natural en python. En el módulo *corpus* tiene incluidos varios corpus para poder trabajar con ellos. Puedes ver la lista de todos ellos en \url{https://www.nltk.org/nltk_data/}. En esta práctica vas a trabajar con un conjunto de tweets. En concreto, se trata de 10 000 tweets separados en dos subconjuntos distintos: 5 000 tweets con sentimiento positivo y 5 000 tweets con sentimiento negativo.

A continuación se muestra el código para obtener estos dos conjuntos por separados. La veriable tweets_positivos es una lista de 5 000 textos, correspondientes a los 5 000 tweets positivos. La variable tweets_negativos es otra lista de 5 000 textos, correspondientes a los 5 000 tweets negativos.

In [2]:
import nltk
#descargar el conjunto de tweets al ordenador
nltk.download('twitter_samples')

from nltk.corpus import twitter_samples

tweets_positivos = twitter_samples.strings('positive_tweets.json')
tweets_negativos = twitter_samples.strings('negative_tweets.json')

[nltk_data] Downloading package twitter_samples to
[nltk_data]     /Users/juancho/nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!


['#FollowFriday @France_Inte @PKuchly57 @Milipol_Paris for being top engaged members in my community this week :)']


Una vez que tienes las dos listas con todos los ejemplos necesarios, vas a separarlos en dos grupos, para utilizarlos como conjuntos de entrenamiento y de test. El conjunto de entrenamiento debe englobar el 80% del total de datos y el conjunto de test debe contener el 20% restante de ejemplos. Además, al partir de un número exactamente igual de ejemplos positivos que de ejemplos negativos, en los dos nuevos conjuntos (entrenamiento y test) las dos clases también tienen que estar balanceadas. Los ejemplos que pertenezcan a cada conjunto se deben asignar de manera aleatoria, es decir, no asignar los primeros positivos y negativos a train, y los últimos de cada clase a test. Recuerda fijar la semilla de la aleatoriedad, para que el experimento sea reproducible.

Estos conjuntos los vas a completar con dos arrays de numpy que indiquen, para cada ejemplo del conjunto, si tiene un sentimiento positivo (clase 1) o un sentimiento negativo (clase 0).

Es decir, debes crear 4 variables nuevas:
* train: lista con el 80% de los datos originales. La mitad de ellos tienen que ser positivos y la otra mitad negativos.
* train_y: array de numpy de ceros y unos. En las posiciones donde haya un tweet positivo, en este array se almacena un 1. En las posiciones donde haya un tweet negativo, en este array se almacena un 0.
* test: lista con el 20% de los datos originales no incluidos en train. La mitad de ellos tienen que ser positivos y la otra mitad negativos.
* test_y: array de numpy de ceros y unos. En las posiciones donde haya un tweet positivo, en este array se almacena un 1. En las posiciones donde haya un tweet negativo, en este array se almacena un 0.

In [40]:
from sklearn.model_selection import train_test_split
import numpy as np


X = tweets_positivos + tweets_negativos
y = np.concatenate((np.ones(len(tweets_positivos)),np.zeros(len(tweets_negativos))))

train, test, train_y, test_y = train_test_split(X,y,test_size=0.2, random_state=12, stratify=y)


### Clasificación mediante regresión logística
Una vez que ya tienes los conjuntos de entrenamiento y de test preparados, vas a codificar cada una de las palabras existentes en el conjunto de entrenamiento. Para comenzar, debes crear la tabla de frecuencias, en la que se mide cuántas veces aparece cada palabra de los tweets de entrenamiento en la clase positiva, y cuántas veces aparece en la clase negativa. Se recomienda utilizar un diccionario para este propósito.

Ten en cuenta que cada tweet lo tienes almacenado como un string. Para tratar cada palabra por separado puedes utilizar el método split. Por defecto, este método separa es string por cada espacio en blanco que encuentra, por lo que obtendrás una lista de palabras.

In [41]:
frecuencias = {}
for tweet, y in zip(train, train_y):

    palabras = tweet.split()

    for palabra in palabras:

        clave = (palabra,y)

        frecuencias[clave] = frecuencias.get(clave,0) + 1

Utilizando el diccionario de frecuencias que ya has creado, crea una función que sea capaz de codificar un tweet en un array (de 1 fila y 3 columnas) con tres números:
* el primer número siempre es 1, para el bias de la regresión
* el segundo número será la suma de las frecuencias positivas de todas las palabras del tweet
* el tercer número será la suma de las frecuencias negativas de todas las palabras del tweet

In [42]:
def codifica_tweet(tweet, frecuencias):
    x = np.ones((1,3))
    palabras = tweet.split()

    for palabra in palabras:
        x[0,1] += frecuencias.get((palabra,1.0),0)
        x[0,2] += frecuencias.get((palabra,0.0),0)
    
    return x

Utilizando la función anterior, convierte el conjunto de train (la parte de las características) en un array de numpy con tantas filas como ejemplos tenga el conjunto y con tres columnas

In [ ]:
train_X = np.zeros((len(train),3))

for i,tweet in enumerate(train):

    train_X[i,:] = codifica_tweet(tweet, frecuencias)


print(train_X.shape)

(8000, 3)


Mediante este conjunto que has creado  las clases correspondientes a cada ejemplo que tienes almacenadas en la variable train_y, ya puedes entrenar el modelo de regresión logística para que aprenda los pesos del modelo.

In [45]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(fit_intercept=False)

model.fit(train_X,train_y)


LogisticRegression(fit_intercept=False)

Con el modelo entrenado, puedes pasar a predecir cada uno de los ejemplos de test y obtener el *accuracy* de esa predicción.

In [47]:
test_X = np.zeros((len(test),3))

for i,tweet in enumerate(test):

    test_X[i,:] = codifica_tweet(tweet, frecuencias)

from sklearn.metrics import accuracy_score

y_pred = model.predict(test_X)
accuracy = accuracy_score(test_y, y_pred)

print(f"Accuracy: {accuracy}")

Accuracy: 0.958


### Clasificación mediante Naive Bayes
Vas a utilizar el mismo conjunto de datos que ya has dividido en entrenamiento y test, pero en este caso vas a entrenar un modelo de Naive Bayes para clasificar los tweets como positivos o negativos.

Partimos del diccionario de frecuencias que has creado en el apartado anterior. Con él, los tweets del conjunto de entrenamiento y sus clases asociadas, puedes entrenar el modelo.

Con el modelo entrenado, puedes calcular el accuracy sobre el conjunto de test.